In [0]:
dbutils.widgets.text(name='s3_path', defaultValue='', label='S3 Path')
s3_path = dbutils.widgets.get('s3_path')
dbutils.widgets.text(name='catalog', defaultValue='', label='catalog')
catalog_name = dbutils.widgets.get('catalog')
dbutils.widgets.text(name='schema', defaultValue='', label='schema')
schema_name = dbutils.widgets.get('schema')
dbutils.widgets.text(name='volume_name', defaultValue='', label='volume_name')
volume_name = dbutils.widgets.get('volume_name')

In [0]:
if catalog_name not in spark.sql("select collect_set(catalog_name) from system.information_schema.catalogs").first()[0]:
  raise Exception(f"catalog {catalog_name} does not exist")

In [0]:
if schema_name not in spark.sql(f"select collect_set(schema_name) from system.information_schema.schemata where catalog_name = '{catalog_name}'").first()[0]:
  raise Exception(f"Schema {schema_name} does not exist in catalog {catalog_name}")

In [0]:
if volume_name in spark.sql(f"select collect_set(volume_name) from system.information_schema.volumes where volume_catalog = '{catalog_name}' and volume_schema = '{schema_name}' and volume_name = '{volume_name}'").first()[0]:
  raise Exception(f"Volume {volume_name} already exists in {catalog_name}.{schema_name}")

In [0]:
managed = True if not s3_path else False
s3_path = s3_path.replace('s3a://','s3://')

In [0]:
if s3_path:
  dbutils.fs.ls(s3_path)

In [0]:
if s3_path in spark.sql(f"select collect_set(storage_location) from system.information_schema.volumes where storage_location = '{s3_path}'").first()[0]:
  volume_error = spark.sql(f"select concat(volume_catalog,'.',volume_schema,'.',volume_name) from system.information_schema.volumes where storage_location = '{s3_path}'").first()[0]
  raise Exception(f"The path {s3_path} is already in  use in Volume {volume_error}")

In [0]:

if managed:
  external = ""
  location = ""
  if not volume_name:
    raise Exception(f"{volume_name} parameter is required in managed Volume")
else:
  external = "EXTERNAL"
  location = f"LOCATION '{s3_path}'"
  if not volume_name:
    volume_name = spark.sql(
    """
    SELECT
      regexp_replace(
        regexp_replace(
          regexp_replace(
            regexp_replace(
              regexp_replace(
                regexp_replace(
                  lower(:s3_path),
                  's3://',
                  ''
                ),
                '_',
                '-'
              ),
              '/',
              '-'
            ),
            'serasaexperian-ecs-datalakehouse-prd-',
            ''
          ),
          'serasaexperian-ecs-ml-prd-sagemaker-stage',
          'sagemaker-stage'
        ),
        '-$',
        ''
      ) AS volume_name
    """,
    args={"s3_path": s3_path}
).first()[0]

In [0]:
query = f"""
  CREATE {external} VOLUME IF NOT EXISTS {catalog_name}.{schema_name}.`{volume_name}`
  {location} COMMENT 'Volume {volume_name} in {catalog_name} for {schema_name} purposes. {"LOCATION: " + s3_path if external else ""}'
"""
print(query)

In [0]:
spark.sql(query)